In [11]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import os, random
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Specify the directories and layer inresults_dirtion
results_dir = '../Results/'

In [3]:
target_img_info = pd.read_csv((os.path.join(results_dir, 'selected_imgs_sEEG.csv')))

In [4]:
filler_img_info = pd.read_csv(os.path.join(results_dir, 'filler_imgs_sEEG.csv'))

In [5]:
# Specify the parameters
n_imgs_per_cond = 15
cond_dict = {1:'Large RE, Small Dist',
            2:'Large RE, Large Dist',
            3:'Small RE, Small Dist',
            4:'Small RE, Large Dist'}
trials_per_block = 150
total_blocks = 13

In [27]:
repeated_fillers = filler_img_info.iloc[:5*total_blocks]
nonrepeat_fillers = filler_img_info.iloc[5*total_blocks:].reset_index(drop=True)

In [31]:
repeated_fillers

,Unnamed: 0,Index,Image type,Image index,Distinctiveness,Reconstruction error,Distinctiveness rank,RE rank
0,0,Target-13,Target,13,1028.490965,0.048492,5528.0,5595.0
1,1,Target-35,Target,35,950.452337,0.044604,4248.0,4533.0
2,2,Target-47,Target,47,1054.453721,0.048502,5967.0,5598.0
3,3,Target-84,Target,84,960.994005,0.045019,4425.0,4666.0
4,4,Target-120,Target,120,960.948838,0.046065,4421.0,4929.0
...,...,...,...,...,...,...,...,...
60,60,Target-1649,Target,1649,1023.126735,0.047184,5454.0,5257.0
61,61,Target-1688,Target,1688,1052.879704,0.049151,5939.0,5753.0
62,62,Target-1715,Target,1715,1037.490198,0.046154,5679.0,4956.0
63,63,Target-1720,Target,1720,1040.637973,0.048091,5740.0,5505.0


In [7]:
# Sort the images into different groups
Large_RE_Small_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Small')].sort_values(by='Reconstruction error', ascending=False).reset_index(drop=True)
Large_RE_Large_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Large')].sort_values(by='Reconstruction error', ascending=False).reset_index(drop=True)
Small_RE_Small_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Small')].sort_values(by='Reconstruction error').reset_index(drop=True)
Small_RE_Large_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Large')].sort_values(by='Reconstruction error').reset_index(drop=True)

In [8]:
img_cond_dict = {1:Large_RE_Small_Dist,
                2:Large_RE_Large_Dist,
                3:Small_RE_Small_Dist,
                4:Small_RE_Large_Dist}

1. Memorability effect is quite robust at different memory lags. Maybe let's just try to repeat the target images at lages of 20-40. We should make sure that the mean repetition lag doesn't differ across the four groups of images.
2. We can have filler images (i.e. those that are not part of the four groups) that repeat at a shorter timescale (1-3), as an attention check.
3. Because it may be tiring to do the task all in one go, we can break the task in to 4 blocks of 7 minutes each.
Each block consists of 17 images from each of the target image group (therefore 68 in total), 5 filler images that repeat at short timescle, and 30 fller images that never repeat. This leads to a total of 176 trials each.
4. Following Lin et al. (2021, Cognition), each image will be presented for 1.2 s and ITI is 1.2
Therefore, each block will last about 7 mins (422.4 seconds).
5. Filler images can be sampled from the filler_images dataframe, which are images with RE and Dist in the middle.
The experiment dataframe should consist of the following columns: block number, trial number, image index, trial type (old or new), response (empty for now) and reaction time (empty for now)

In [52]:
def generate_trial_sequence(
    num_conditions=4,
    images_per_condition=15,
    target_repeat_lag_range=(20, 40),
    num_filler_single=20,
    num_filler_repeat=5,
    filler_repeat_lag_range=(1, 3),
    total_trials=150,
    seed=None,
    max_repeats_in_last_n=6,
    check_last_n=10
):
    if seed is not None:
        random.seed(seed)

    # 1. Generate image labels
    target_images = [
        f'T{cond}_{i}' for cond in range(1, num_conditions + 1)
        for i in range(1, images_per_condition + 1)
    ]
    filler_single = [f'F_S{i}' for i in range(1, num_filler_single + 1)]
    filler_repeat = [f'F_R{i}' for i in range(1, num_filler_repeat + 1)]

    sequence = [None] * total_trials
    used_positions = set()

    def place_with_lag(img, min_lag, max_lag, total_len):
        attempts = 0
        while attempts < 1000:
            pos1 = random.randint(0, total_len - min_lag - 1)
            lag = random.randint(min_lag, max_lag)
            pos2 = pos1 + lag
            if pos2 >= total_len:
                attempts += 1
                continue
            if pos1 not in used_positions and pos2 not in used_positions:
                return pos1, pos2
            attempts += 1
        return None, None

    # 2. Place target images
    for img in target_images:
        pos1, pos2 = place_with_lag(img, *target_repeat_lag_range, total_trials)
        if pos1 is None:
            raise RuntimeError(f"Could not place target {img}")
        sequence[pos1] = (img, False)
        sequence[pos2] = (img, True)
        used_positions.update([pos1, pos2])

    # 3. Place repeating fillers
    for img in filler_repeat:
        pos1, pos2 = place_with_lag(img, *filler_repeat_lag_range, total_trials)
        if pos1 is None:
            raise RuntimeError(f"Could not place repeating filler {img}")
        sequence[pos1] = (img, False)
        sequence[pos2] = (img, True)
        used_positions.update([pos1, pos2])

    # 4. Place non-repeating fillers
    remaining_slots = [i for i in range(total_trials) if i not in used_positions]
    if len(remaining_slots) < num_filler_single:
        raise RuntimeError("Not enough slots for single fillers")
    random.shuffle(remaining_slots)
    for i, img in enumerate(filler_single):
        pos = remaining_slots[i]
        sequence[pos] = (img, False)
        used_positions.add(pos)

    # 5. Post-hoc check: Too many repeats at the end?
    def count_repeats(trial_list, start):
        return sum(1 for t in trial_list[start:] if t and t[1] is True)

    last_n_start = total_trials - check_last_n
    repeat_count = count_repeats(sequence, last_n_start)

    if repeat_count > max_repeats_in_last_n:
        # Try to reduce by swapping
        early_idxs = [i for i in range(total_trials - check_last_n) if sequence[i] and sequence[i][1] is False]
        late_idxs = [i for i in range(last_n_start, total_trials) if sequence[i] and sequence[i][1] is True]

        swaps_done = 0
        for late_idx in late_idxs:
            for early_idx in early_idxs:
                # Swap early non-repeat with late repeat
                sequence[early_idx], sequence[late_idx] = sequence[late_idx], sequence[early_idx]
                new_repeat_count = count_repeats(sequence, last_n_start)
                if new_repeat_count <= max_repeats_in_last_n:
                    swaps_done += 1
                    break
                else:
                    # Revert and try next
                    sequence[early_idx], sequence[late_idx] = sequence[late_idx], sequence[early_idx]
            if count_repeats(sequence, last_n_start) <= max_repeats_in_last_n:
                break

        # Final check
        if count_repeats(sequence, last_n_start) > max_repeats_in_last_n:
            raise RuntimeError("Could not enforce repeat constraint in last trials")

    # 6. Format output
    final_sequence = []
    for i, trial in enumerate(sequence):
        if trial is None:
            final_sequence.append({
                'trial_num': i + 1,
                'image': None,
                'is_repeat': None,
                'type': 'EMPTY'
            })
        else:
            img, is_repeat = trial
            trial_type = 'target' if img.startswith('T') else 'filler'
            final_sequence.append({
                'trial_num': i + 1,
                'image': img,
                'is_repeat': is_repeat,
                'type': trial_type
            })

    return final_sequence


In [53]:
def generate_valid_sequence(max_attempts=100, verbose=True):
    for attempt in range(max_attempts):
        seed = random.randint(0, 1000000)
        try:
            sequence = generate_trial_sequence(seed=seed)
            if verbose:
                print(f"✅ Sequence generated successfully on attempt {attempt+1} (seed={seed})")
            return sequence
        except RuntimeError as e:
            if verbose:
                print(f"⚠️ Attempt {attempt+1} failed (seed={seed}): {str(e)}")
    raise RuntimeError("❌ Failed to generate a valid sequence after multiple attempts.")


In [54]:
for curr_block in range(total_blocks):
    # Generate trial sequence
    
    block_sequence_dict = generate_valid_sequence()
    
    # Place the images
    if curr_block != 12:
        df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_f_single = nonrepeat_fillers.iloc[curr_block*20:(curr_block+1)*20]
        df_f_repeat = repeated_fillers.iloc[curr_block*5:(curr_block+1)*5]
    else:
        df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:]
        df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:]
        df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:]
        df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:]
        df_f_single = nonrepeat_fillers.iloc[curr_block*20:]
        df_f_repeat = repeated_fillers.iloc[curr_block*5:]
    
    # Step 1: Build a lookup dictionary for images
    image_lookup = {}
    # Target conditions
    for i, df in enumerate([df_t1, df_t2, df_t3, df_t4], start=1):
        for j, path in enumerate(df['Index']):
            label = f'T{i}_{j+1}'  # matches label from generate_trial_sequence
            image_lookup[label] = path

    # Fillers - single appearance
    for i, path in enumerate(df_f_single['Index'], start=1):
        label = f'F_S{i}'
        image_lookup[label] = path

    # Fillers - repeating
    for i, path in enumerate(df_f_repeat['Index'], start=1):
        label = f'F_R{i}'
        image_lookup[label] = path

    # Step 2: Assign image paths to your generated sequence

    for trial in block_sequence_dict:
        label = trial['image']
        trial['Image Index'] = image_lookup.get(label, None)
    
    block_df = pd.DataFrame(block_sequence_dict)
    block_df['Block'] = curr_block+1
    

⚠️ Attempt 1 failed (seed=997588): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on attempt 2 (seed=486324)
✅ Sequence generated successfully on attempt 1 (seed=370839)
✅ Sequence generated successfully on attempt 1 (seed=130177)
✅ Sequence generated successfully on attempt 1 (seed=374026)
✅ Sequence generated successfully on attempt 1 (seed=87490)
✅ Sequence generated successfully on attempt 1 (seed=138823)
✅ Sequence generated successfully on attempt 1 (seed=139445)
⚠️ Attempt 1 failed (seed=736584): Could not enforce repeat constraint in last trials
⚠️ Attempt 2 failed (seed=400545): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on attempt 3 (seed=209070)
✅ Sequence generated successfully on attempt 1 (seed=926597)
✅ Sequence generated successfully on attempt 1 (seed=624854)
✅ Sequence generated successfully on attempt 1 (seed=922971)
✅ Sequence generated successfully on attempt 1 (seed=906229)
✅ Sequence 

In [61]:
Exp_dir = './Experiment/'
for sub in range(1, 51):
    sub_dir = os.path.join(Exp_dir, f"sub-{sub:02d}")
    if not os.path.isdir(sub_dir):
        os.mkdir(sub_dir)
    for curr_block in range(total_blocks):
        # Generate trial sequence
        block_sequence_dict = generate_valid_sequence()

        # Place the images
        if curr_block != 12:
            df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_f_single = nonrepeat_fillers.iloc[curr_block*20:(curr_block+1)*20]
            df_f_repeat = repeated_fillers.iloc[curr_block*5:(curr_block+1)*5]
        else:
            df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:]
            df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:]
            df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:]
            df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:]
            df_f_single = nonrepeat_fillers.iloc[curr_block*20:]
            df_f_repeat = repeated_fillers.iloc[curr_block*5:]

        # Step 1: Build a lookup dictionary for images
        image_lookup = {}
        # Target conditions
        for i, df in enumerate([df_t1, df_t2, df_t3, df_t4], start=1):
            for j, path in enumerate(df['Index']):
                label = f'T{i}_{j+1}'  # matches label from generate_trial_sequence
                image_lookup[label] = path

        # Fillers - single appearance
        for i, path in enumerate(df_f_single['Index'], start=1):
            label = f'F_S{i}'
            image_lookup[label] = path

        # Fillers - repeating
        for i, path in enumerate(df_f_repeat['Index'], start=1):
            label = f'F_R{i}'
            image_lookup[label] = path

        # Step 2: Assign image paths to your generated sequence

        for trial in block_sequence_dict:
            label = trial['image']
            trial['Image Index'] = image_lookup.get(label, None)

        block_df = pd.DataFrame(block_sequence_dict)
        block_df['Block'] = curr_block+1
        
        block_dir = os.path.join(sub_dir, f"Block{curr_block+1:02d}_info.csv")
        block_df.to_csv(block_dir)

⚠️ Attempt 1 failed (seed=7288): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on attempt 2 (seed=801151)
⚠️ Attempt 1 failed (seed=243221): Could not enforce repeat constraint in last trials
⚠️ Attempt 2 failed (seed=6039): Could not enforce repeat constraint in last trials
⚠️ Attempt 3 failed (seed=330106): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on attempt 4 (seed=836209)
✅ Sequence generated successfully on attempt 1 (seed=505077)
✅ Sequence generated successfully on attempt 1 (seed=573619)
⚠️ Attempt 1 failed (seed=377466): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on attempt 2 (seed=425904)
⚠️ Attempt 1 failed (seed=940861): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on attempt 2 (seed=678650)
⚠️ Attempt 1 failed (seed=597285): Could not enforce repeat constraint in last trials
✅ Sequence generated successfully on

In [59]:
sub_dir

'./Experiment/sub-50'

In [28]:
block1_sequence_df['Image placeholder index'] = [int(k.split('_')[1]) if k[0]=='T' else int(k[3:]) for k in block1_sequence_df['image']]

In [30]:
df_t1 = img_cond_dict[1].iloc[:n_imgs_per_cond]
df_t2 = img_cond_dict[2].iloc[:n_imgs_per_cond]
df_t3 = img_cond_dict[3].iloc[:n_imgs_per_cond]
df_t4 = img_cond_dict[4].iloc[:n_imgs_per_cond]
df_f_single = nonrepeat_fillers.iloc[:20]
df_f_repeat = repeated_fillers.iloc[:5]

In [34]:
# Step 1: Build a lookup dictionary for images

image_lookup = {}
# Target conditions
for i, df in enumerate([df_t1, df_t2, df_t3, df_t4], start=1):
    for j, path in enumerate(df['Index']):
        label = f'T{i}_{j+1}'  # matches label from generate_trial_sequence
        image_lookup[label] = path

# Fillers - single appearance
for i, path in enumerate(df_f_single['Index'], start=1):
    label = f'F_S{i}'
    image_lookup[label] = path

# Fillers - repeating
for i, path in enumerate(df_f_repeat['Index'], start=1):
    label = f'F_R{i}'
    image_lookup[label] = path

# Step 2: Assign image paths to your generated sequence

# Let's say you already called:
# sequence = generate_trial_sequence(seed=42)

for trial in block1_sequence_dict:
    label = trial['image']
    trial['Image Index'] = image_lookup.get(label, None)

In [35]:
block1_sequence_dict

[{'trial_num': 1,
  'image': 'T2_6',
  'is_repeat': True,
  'type': 'target',
  'Image Index': 'Filler-2883'},
 {'trial_num': 2,
  'image': 'T4_1',
  'is_repeat': False,
  'type': 'target',
  'Image Index': 'Filler-5957'},
 {'trial_num': 3,
  'image': 'T1_6',
  'is_repeat': False,
  'type': 'target',
  'Image Index': 'Filler-6863'},
 {'trial_num': 4,
  'image': 'F_R3',
  'is_repeat': False,
  'type': 'filler',
  'Image Index': 'Target-47'},
 {'trial_num': 5,
  'image': 'T1_1',
  'is_repeat': False,
  'type': 'target',
  'Image Index': 'Filler-6188'},
 {'trial_num': 6,
  'image': 'T2_3',
  'is_repeat': False,
  'type': 'target',
  'Image Index': 'Filler-5531'},
 {'trial_num': 7,
  'image': 'F_R3',
  'is_repeat': True,
  'type': 'filler',
  'Image Index': 'Target-47'},
 {'trial_num': 8,
  'image': 'F_R4',
  'is_repeat': False,
  'type': 'filler',
  'Image Index': 'Target-84'},
 {'trial_num': 9,
  'image': 'T4_2',
  'is_repeat': False,
  'type': 'target',
  'Image Index': 'Filler-2138'},
